In [25]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from pandas import Index
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import request


In [26]:
def signal_process(table,event_var='PM25_Marseille_Longchamp',event_threshold = 40, duration = 6,var_name_list='air pollution'):
    """Find the spike in pm25 and plot a sliding window for google trend """

    index_list=np.where(table[event_var]>event_threshold)[0]
    
    # add first element
    shrink_index_list = [index_list[0]]
    date_set ={table.date[index_list[0]]}
    
    # exclude for the same day
    for cnt in range(1,len(index_list)):
        if table.date[index_list[cnt]] in date_set:
            pass
        else:
            shrink_index_list.append(index_list[cnt].copy())
            date_set.add(table.date[index_list[cnt]])
    
    bind_list = []
    for start_index in shrink_index_list:
        this_sequence = table.loc[range(start_index-2,start_index+duration),
                                  var_name_list]
        start_date = table.datetime[start_index]
        tuples = list(zip([start_date]*(duration+2),range(-2,duration)))

        this_sequence.index = pd.MultiIndex.from_tuples(tuples,names=['event_date_index','hour_index'])
    #         this_sequence = (this_sequence- this_sequence.mean())/this_sequence.std()

        bind_list.append(this_sequence)
    new_table=pd.concat(bind_list)     
    
    return new_table

In [40]:
def fetching_trend(airdata, kw_list,location_code='FR-U',timezone = 0):
    input_table = airdata
    newest_datetime   = max(input_table['datetime'])
    earliest_datetime = min(input_table['datetime'])
    pytrends = TrendReq(hl='En-US', tz=timezone)
    print("fectching trend ")
    ggtrend_list =[]
    for keyword in kw_list:
        
        this_list = [keyword]
        try: 
            topic_code = pytrends.suggestions(keyword)[0]['mid']
            this_list.append(topic_code)
        except IndexError:
            pass
            
        ggtrend = pytrends.get_historical_interest(this_list, 
                                 year_start=earliest_datetime.year,
                                          month_start=earliest_datetime.month, 
                                          day_start=earliest_datetime.days_in_month, 
                                          hour_start=earliest_datetime.hour, 
                                 year_end=newest_datetime.year,
                                          month_end=newest_datetime.month, 
                                          day_end=newest_datetime.days_in_month, 
                                          hour_end=newest_datetime.hour, 
                                 cat=0, # all categorical
                                          geo=location_code,  # Provence-Alpes-Côte d'Azur
                                          gprop='',  # no specific group # image #hastag etc
                                          sleep=0)
        ggtrend.columns= Index([keyword, keyword+" topic", 'isPartial'], dtype='object')
        ggtrend_list.append(ggtrend)
    print("trend is fectched")
    return ggtrend_list

In [28]:
# loading table 

In [29]:
parca_table= pd.read_csv("../stage/parif.csv")

In [30]:
parca_table['datetime']=parca_table['datetime'].apply(lambda string: datetime.strptime(string,'%Y-%m-%d %H:%M:%S'))

In [31]:
full_date_time = pd.date_range(parca_table[['datetime']].min()[0],parca_table[['datetime']].max()[0],freq='H')
full_time_set = set(map(lambda mytime: mytime.strftime("%Y-%m-%d %H:%M:%S"),full_date_time.to_series())) 
data_time_set = set(map(lambda timestamp: str(timestamp),parca_table['datetime']))
    # pad na values
if len(full_time_set) != len(data_time_set):
    pad_time = full_time_set -data_time_set
    pad_table = pd.DataFrame()
    pad_table['datetime']=[datetime.strptime(mytime,"%Y-%m-%d %H:%M:%S") for mytime in pad_time]
    for column in parca_table.columns:
        if column !="datetime":
            pad_table[column]=None
    parca_table = pd.concat([parca_table,pad_table],ignore_index=True)

In [32]:
# fetching google trend

In [41]:
KW_LIST= ['difficultés respiratoires',
          'difficulté à respirer',
          'asthme',
          'difficulté pour respirer',
          'sensation d’étouffement',
          'dyspnée',
          'spasmophilie',
          'nuage de pollution',
          'saleté de l’air',
          'particules nocives',
          'respiration',
          'maladies respiratoires',
          'pollution de l’air',
          'pollution atmosphérique',
          'pollution',
          'Masque', #mask
          "Purificateur d'air", #air purifier
          
         ]

In [42]:
LOCATION_CODE = 'FR-J'  # for Île-de-France

In [43]:
ggtrend_list = fetching_trend(parca_table,kw_list=KW_LIST,location_code=LOCATION_CODE,timezone = 0)
ggtrend=pd.concat(ggtrend_list,axis=1)
ggtrend.reset_index(inplace=True)

fectching trend 
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with c

ValueError: Length mismatch: Expected axis has 0 elements, new values have 3 elements

In [ ]:
merged_t=parca_table.merge(ggtrend,left_on='datetime',right_on='date')
kw_str  ="_".join(KW_LIST)
merged_t.to_csv("../stage/parif_{}_{}.csv".format(kw_str,parca_table.columns[1]))

In [ ]:
merged_t.head()

In [ ]:
signal_table=signal_process(merged_t,event_var='PA04C_PM25',event_threshold = 35,duration = 12,var_name_list=list(ggtrend.columns)+['PA04C_PM25'])

In [ ]:
signal_table.groupby("hour_index").mean().plot()

In [16]:
signal_table.to_csv("../output/air_parif_threshold_35_colsum_duration_12H.csv")